In [ ]:
# - If you want to run this notebook independently,
# run this cell w/ the below line commented out
%%script false --no-raise-error

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import xarray as xr

# - Set data path
dpath = '/opt/acoustic-variability/data/'

# - Set saved out figure path
figpath = '/opt/acoustic-variability/clean_version/figures/'

In [3]:
%run create_JASADCP_metadata_df.ipynb

# of files w/o bandwidth: 57


In [12]:
# - How many files for each type of instrument?
print(df['instrument_name'].value_counts())

NB-VM-150         1088
OS-38              519
OS-75              363
WH-300             188
OS-150              26
NB-VM-300           25
NB-VM-75            14
BB-VM-150           13
OS-II-38             6
WH-Mariner-300       3
NB-DR-150            3
DCP4400A             1
WH-Mariner-600       1
WH-1200              1
OS-II-75             1
Name: instrument_name, dtype: int64


# Mean volume backscattering strength eqn **(EQUATION 1)**:  
$S_v = \color{red}{C} + \color{blue}{10log((T_x + 273.16)*R^2)} - \color{orange}{L_{DBM}} - \color{green}{P_{DBW}} + \color{purple}{2 \alpha R} + 10log(10^{k_c(E-E_r)/10} - 1)$, where:  
- C = constant combining several params specific to each instrument
- $T_x$ = temperature measured at the transducer ($^{\circ}$C)
- R = along-beam range to the measurement, taken in the last quarter of the bin for Workhorse, Long Ranger, and Quartermaster, and at midpt of the bin for other instruments
- $L_{DBM}$ = 10log(transmit pulse length, meters)
- $P_{DBW}$ = 10log(transmit power, Watts)
- $\alpha$ = absorption coefficient of the water
- $k_c$ = RSSI slope (dB/count)
- $E_r$ = noise floor (counts)
- Units = dB, referenced to $[meters × 4π]^{-1}$
- From Mullison (2017), meant for newer generation ADCPs
- Colors correspond to terms in eqn 2 below

# Mean volume backscattering strength eqn (FST-003) **(EQUATION 2)**:  
$S_v = \color{red}{10log(\frac{4.47 \times 10^{-20} K_2 K_S}{c})} + \color{blue}{10log((T_x + 273.16)*R^2)} - \color{orange}{10log(P)} - \color{green}{10log(K_1)} + \color{purple}{2 \alpha R} + 10log(10^{k_c(E-E_r)/10} - 1)$
- $K_2$ = system noise factor (dimensionless)
- $K_S$ = system constant, depends on NBADCP frequency
- c = speed of sound at the scattering layer being measured
- P = transmit pulse length
- $K_1$ = real-time power into the water (Watts) 
- All others the same as Mullison (2017) eqn
- Units = dB, referenced to $[meters × 4π]^{-1}$
- From FST-003 by TRDI (1998), meant for NBADCPs technically
- Colors correspond to terms in eqn 1 above

# NBADCP-specific values and calculations (FST-003):

**$C$: constant to match eqn 1 to 2 (dB)**  
$C = 10log[\frac{4.47 x 10^{-20} K_2 K_s}{c}]$, where c = speed of sound at the scattering layer being measured

**$K_2$: system noise factor (dimensionless)**

In [ ]:
# - From FST-003, pg 6:
NB_K2_values = {
    'instrument_prefix' : ['NB-VM','NB-VM','NB-VM','NB-VM','NB-VM','NB-DR','NB-DR','NB-DR','NB-DR','NB-DR'],
    'freq_round_kHz' : [75, 150, 300, 600, 1200, 75, 150, 300, 600, 1200],
    'K2' : [2.5, 4.3, 4.5, 9.1, 10.5, 2.2, 3.6, 4.2, 7.1, 8.1]
}
NB_K2_values = pd.DataFrame(NB_K2_values)
NB_K2_values

**$K_S$: system constant, depends on NBADCP freq**

In [6]:
# - From FST-003, pg 10:
NB_Ks_values = {
    'freq_round_kHz' : [75, 150, 300, 600, 1200],
    'Ks' : [1.09E5, 4.17E5, 7.69E5, 1.56E6, 5.65E6]
}
NB_Ks_values = pd.DataFrame(NB_Ks_values)
NB_Ks_values

,freq_round_kHz,Ks
0,75,109000.0
1,150,417000.0
2,300,769000.0
3,600,1560000.0
4,1200,5650000.0


**$K_1$: real-time power into the water (Watts)**

$K_1 = [\frac{(V_s \times a)-b}{c}]^2 \times K_{1c}$
- $V_s$ = real-time supply voltage to NBADCP transmitter (counts, V)
- a, b, c constants shown in table below
- $K_{1c}$ = power into the water during calibration at RDI factory (Watts), see table below

In [7]:
NB_abc_values = {
    'instrument_prefix_voltage' : ['NB-DR', 'NB-VM-110', 'NB-VM-220', 'NB-SC'],
    'a' : [1.397, 1.397, 0.699, 0.170],
    'b' : [6.30, 4.27, 4.27, 5.76],
    'c' : [35.10, 37.14, 37.14, 34.24]
}
NB_abc_values = pd.DataFrame(NB_abc_values)
NB_abc_values

,instrument_prefix_voltage,a,b,c
0,NB-DR,1.397,6.30,35.10
1,NB-VM-110,1.397,4.27,37.14
2,NB-VM-220,0.699,4.27,37.14
3,NB-SC,0.170,5.76,34.24


**$K_{1c}$: power into the water during calibration at RDI factory (Watts)**

In [8]:
# - From FST-003, pg 10:
# K1c = units of Watts
NB_K1c_values = {
    'instrument_prefix' : ['NB-VM','NB-VM','NB-VM','NB-VM','NB-VM','NB-DR','NB-DR','NB-DR','NB-DR','NB-DR'],
    'freq_round_kHz' : [75, 150, 300, 600, 1200, 75, 150, 300, 600, 1200],
    'K1c' : [5.5, 3.3, 3.6, 1.7, 1.2, 6.1, 3.9, 4.1, 1.9, 1.3]
}
NB_K1c_values = pd.DataFrame(NB_K1c_values)
NB_K1c_values

,instrument_prefix,freq_round_kHz,K1c
0,NB-VM,75,5.5
1,NB-VM,150,3.3
2,NB-VM,300,3.6
3,NB-VM,600,1.7
4,NB-VM,1200,1.2
5,NB-DR,75,6.1
6,NB-DR,150,3.9
7,NB-DR,300,4.1
8,NB-DR,600,1.9
9,NB-DR,1200,1.3


**$K_c$: conversion factor for echo intensity (dB/counts)**

- For E < 200 counts, $K_c = \frac{127.3}{T_e+271}$  
- If E is 200-230 counts, calibration of $K_c$ must be done at RDI.
- If E > 230 counts, $K_c$ can't be calibrated.  
- $T_e$ = temperature of system electronics
- "$T_e$ is used to calculate $K_c$ and $E_r$. $E_r$...is particularly sensitive to changes in $T_e$, so it is crucial to obtain an accurate record of $T_e$."  
- "For DR-NBADCPs in which both system electronics and transducer assembly are immersed, $T_x$ can be substituted for $T_e$."  
- "The ambient temp for the sm electronics will generally differ from the temp recorded at the transducer. Therefore, you must independently measure and record the temperature of the system electronics."

**$E_r$: real-time reference level for echo intensity (counts)**

- "Because the reference level for each beam is sensitive to the temperature of the NBADCP electronics, the calibrated reference value $E_{rc}$ must be adjusted for real-time temperatures before being applied...as $E_r$."
- 3 methods are available: equation method, approximation method, in-situ method
- But I can only really use the in-situ method b/c I don't have values for $T_{ec}$ or $T_{xc}$
    - $E_{rc}$ = reference thermal noise of NBADCP electronics during calibration in counts
    - $T_{ec}$ = temp of sm electronics during calibration of $E_{rc}$ in $^{\circ}$C
    - $T_{xc}$ = temp of transducer during calibration of $E_{rc}$ in $^{\circ}$C
- "In-situ method - the value for $E_r$ can be obtained from the dataset if the following conditions are met:
    - a) Interfering signals that mask the thermal noise level are not present (see step 2).
    - b) The range of the profile is long enough so propagation losses reduce the signal-to-noise ratio well below unity. At this point, the output in counts for echo intensity has reached and remains at its minimum value, which is $E_r$."

# OS-specific values and calculations:

In [9]:
# - C, PDBW, rayleigh distance from Mullison (2017);
# Kc, Er from Jerry Mullison emails in Jan 2020
# -->ASSUMPTION FOR NOW: OS and OS-II have same values
# -->ASSUMPTION FOR NOW: C_25% copied from C_6% below
OS_C_PDBW_Kc_Er_values = {
    'instrument_prefix' : ['OS','OS','OS','OS-II','OS-II','OS-II'],
    'freq_round_kHz' : [150, 75, 38, 150, 75, 38],
    # -->ASSUMPTION FOR NOW: C_25% copied from C_6% below
    'C_25%' : [-156.01, -164.26, -172.19, -156.01, -164.26, -172.19],
    'C_6%' : [-156.01, -164.26, -172.19, -156.01, -164.26, -172.19],
    'PDBW_battery' : [None, None, None, None, None, None],
    'PDBW_power_supply' : [21, 24, 24, 21, 24, 24],
    'rayleigh_distance' : [1.62, 3.24, 8.19, 1.62, 3.24, 8.19],
    'Kc_min' : [0.41, 0.36, 0.36, 0.41, 0.36, 0.36],
    'Kc_mean' : [0.42, 0.39, 0.37, 0.42, 0.39, 0.37],
    'Kc_max' : [0.44, 0.42, 0.37, 0.44, 0.42, 0.37],
    'Kc_sigma' : [0.012, 0.019, 0.004, 0.012, 0.019, 0.004],
    'Er_min' : [18, 11, 5, 18, 11, 5],
    'Er_mean' : [22, 19, 14, 22, 19, 14],
    'Er_max' : [28, 25, 33, 28, 25, 33],
    'Er_sigma' : [2.08, 2.93, 4.83, 2.08, 2.93, 4.83]
}
OS_C_PDBW_Kc_Er_values = pd.DataFrame(OS_C_PDBW_Kc_Er_values)
OS_C_PDBW_Kc_Er_values

,instrument_prefix,freq_round_kHz,C_25%,C_6%,PDBW_battery,PDBW_power_supply,rayleigh_distance,Kc_min,Kc_mean,Kc_max,Kc_sigma,Er_min,Er_mean,Er_max,Er_sigma
0,OS,150,None,-156.01,None,21,1.62,0.41,0.42,0.44,0.012,18,22,28,2.08
1,OS,75,None,-164.26,None,24,3.24,0.36,0.39,0.42,0.019,11,19,25,2.93
2,OS,38,None,-172.19,None,24,8.19,0.36,0.37,0.37,0.004,5,14,33,4.83
3,OS-II,150,None,-156.01,None,21,1.62,0.41,0.42,0.44,0.012,18,22,28,2.08
4,OS-II,75,None,-164.26,None,24,3.24,0.36,0.39,0.42,0.019,11,19,25,2.93
5,OS-II,38,None,-172.19,None,24,8.19,0.36,0.37,0.37,0.004,5,14,33,4.83


# WH-specific values and calculations:

In [10]:
# - C, PDBW, rayleigh distance from Mullison (2017);
# Kc from TRDI field service emails in Jan 2020
# (but they didn't specify a frequency)
# -->ASSUMPTION FOR NOW: WH and WH-Mariner have same values (prob good assumption)
# -->ASSUMPTION FOR NOW: Kc same for all frequencies
WH_C_PDBW_Kc_Er_values = {
    'instrument_prefix' : ['WH','WH','WH','WH-Mariner','WH-Mariner','WH-Mariner'],
    'freq_round_kHz' : [1200, 600, 300, 1200, 600, 300],
    'C_25%' : [-129.44, -139.09, -140.87, -129.44, -139.09, -140.87],
    'C_6%' : [-139.57, -149.14, -151.64, -139.57, -149.14, -151.64],
    'PDBW_battery' : [4.8, 9, 14, 4.8, 9, 14],
    'PDBW_power_supply' : [8.3, 12.5, 17.5, 8.3, 12.5, 17.5],
    'rayleigh_distance' : [1.71, 1.75, 0.87, 1.71, 1.75, 0.87],
    'Kc_mean' : [0.42, 0.42, 0.42, 0.42, 0.42, 0.42],
    'Er_mean' : [None, None, None, None, None, None],
}
WH_C_PDBW_Kc_Er_values = pd.DataFrame(WH_C_PDBW_Kc_Er_values)
WH_C_PDBW_Kc_Er_values

,instrument_prefix,freq_round_kHz,C_25%,C_6%,PDBW_battery,PDBW_power_supply,rayleigh_distance,Kc_mean,Er_mean
0,WH,1200,-129.44,-139.57,4.8,8.3,1.71,0.42,None
1,WH,600,-139.09,-149.14,9.0,12.5,1.75,0.42,None
2,WH,300,-140.87,-151.64,14.0,17.5,0.87,0.42,None
3,WH-Mariner,1200,-129.44,-139.57,4.8,8.3,1.71,0.42,None
4,WH-Mariner,600,-139.09,-149.14,9.0,12.5,1.75,0.42,None
5,WH-Mariner,300,-140.87,-151.64,14.0,17.5,0.87,0.42,None


# BBADCP-specific values and calculations:

In [11]:
# - Kc from TRDI field service emails in Jan 2020
# (but they didn't specify a frequency)
BB_C_PDBW_Kc_Er_values = {
    'instrument_prefix' : ['BB-VM'],
    'freq_round_kHz' : [150],
    # -->ASSUMPTION FOR NOW: -156.01 from OS, -140.87 from WH, used avg for BB here
    'C_25%' : [-148],
    'C_6%' : [None],
    'PDBW_battery' : [None],
    # -->ASSUMPTION FOR NOW: 21 from OS 150kHz, 17.5 taken from WH 300kHz, used OS for BB here
    'PDBW_power_supply' : [21],
    'rayleigh_distance' : [None], 
    'Kc_mean' : [0.45],
    'Er_mean' : [None],
}
BB_C_PDBW_Kc_Er_values = pd.DataFrame(BB_C_PDBW_Kc_Er_values)
BB_C_PDBW_Kc_Er_values

,instrument_prefix,freq_round_kHz,C_25%,C_6%,PDBW_battery,PDBW_power_supply,rayleigh_distance,Kc_mean,Er_mean
0,BB-VM,150,None,None,None,None,None,0.45,None


# Calculations common to all ADCP:

## 1.) **$R$: slant range to depth cell (m)**

$R = \frac{B + |(P-D)/2| + (N \times D) + (D/4)}{cos(\theta)} \times \frac{c'}{1475.1}$
**[method 1]**
- B = blank beyond transmit (m) (same as blanking interval?)
- P = transmit pulse length (m)
- D = depth cell length (m)
- N = depth cell number of the scattering layer being measured
- c' = wted avg of sound speed btwn transducer and depth cell (m/s)
- $\theta$ = angle of transducer beams from vertical (degrees)
- NBADCP, BBADCP, and WH sample echo intensity in the last quarter of each depth cell, not the center --> D/4 accts for this
- All other instruments sample at midpt of bin --> remove D/4, I think?
- NBADCP assumes the speed of sound is 1475.1 m/s in dtmning depth cell size; c'/1475.1 m/s corrects for variations from this value
- From FST-003, used in Lee et al. (2004) and Dwinovantyo et al. (2019)

$R = \frac{B + (L+D)/2 + ((N-1) \times D) + (D/4)}{cos(\theta)} \times \frac{c'}{c_1}$
**[method 2]**
- B = blank beyond transmit (m) (same as blanking interval?)
- D = depth cell length (m)
- N = depth cell number of the scattering layer being measured
- c' = wted avg of sound speed btwn transducer and depth cell (m/s)
- $c_1$ = sound speed used by the instrument (m/s)
- $\theta$ = angle of transducer beams from vertical (degrees)
- From Deines (1999)

$R = \frac{B + (L + D + L_a)/2 + D/4 + (n-1)D}{cos(\theta)}$
**[method 3]**
- B = blank length (m)
- D = depth cell length (m)
- n = depth cell number of the scattering layer being measured
- L = transmit pulse length
- $L_a$ = transmit lag distance (m) 
- $\theta$ = angle of transducer beams from vertical (degrees)
- This expression differs from Deines (1999), where the transmit lag distance is not included
- From Gostiaux and van Haren (2010)

$R$ = depth to center of bin
**[method 4]**
- Can just use the depth to the middle of the bin; errors compared to the complicated R formulas are small

$R$ = depth to center of bin/$cos(\theta)$
**[method 5]**
- Jerry Mullison said "for the OS systems you can use the midpoint of the bins for range - still have to divide by cos(Janus) to get the along beam range"

$R$ = depth to center of bin/$cos(\theta) \times \frac{c'}{c_1}$
**[method 6]**
- I added in the sound speed correction, which is technically always needed

## 2.) **$2 \alpha R$: total absorption (dB)**

$2 \alpha R = \frac{2 \alpha_p B}{cos(\theta)} + \sum_{n=1}^{b} \alpha_n$
- b = range cell number
- $\alpha_n = \frac{2 \alpha D}{cos(\theta)}$
- $\alpha$ = absorption coefficient at that depth
- $\alpha_p$ = absorption at the profiler
- B = blank length (m)
- D = depth cell length (m)
- n = depth cell number of the scattering layer being measured
- $\theta$ = angle of transducer beams from vertical (degrees)
- From Deines (1999)

## 3.) **$E_r$: real-time reference level for echo intensity (counts)**  

If $E_r$ isn't specified, use the minimum echo intensity in all of the received data, as in the following papers:
- Lee et al. (2004): "In the present paper, $E_r$ of each beam was chosen as the minimum echo intensity in all of the received data."
- Gostiaux and van Haren (2010): "The term $E_{noise}$ is the tail value of the minimum RSSI profile over the total record."

## 4.) **c: speed of sound (m/s)**  

$c = 1448.96 + 4.591T - 0.05304T^2 + 2.374 \times 10^{-4}T^3 + 1.34(S-35) + 0.0163z + 1.675 \times 10^{-7}z^2 - 0.01025T(S-35) - 7.139 \times 10^{-13}Tz^3$ **[method 1]**
- T = temp in degC
- S = salinity in PSU, parts per thousand
- z = depth in meters  
- Valid for temps from -2-30degC, salinities from 25-40 ppt, depths from 0-8000m
- Std error of the predicted SS is 0.07 m/s
- Above 1000m, $Tz^3$ term is very small and may be ignored
- From Mackenzie (1981) from Simmonds and MacLennan (2008)

$c = 1449.2 + 4.6T - 0.055T^2 + 0.00029T^3 + (1.34-0.01T)(S-35) + 0.016D$ **[method 2]**
- T = temp in degC
- S = salinity in PSU, parts per thousand
- D = depth in meters  
- From Urick (1983) from "ADCP Principles of Operation: A Practical Primer"

$c = 1492.9 + 3(T-10) - 0.006(T-10)^2 - 0.04(T-18)^2 + 1.2(S-35) - 0.01(T-18)(S-35) + D/61$
- T = temp in degC
- S = salinity in PSU, parts per thousand
- D = depth in meters  
- Predicts c correctly to within 0.1 m/s for temps from -2-23degC, salinities from 30-40ppt, depths above 500m
- Less accurate, but simpler formula good enough for many fishery applications
- From Leroy (1969) from Simmonds and MacLennan (2008)

## 5.) **$\alpha$: absorption coefficient (dB/km)**

$\alpha = \frac{A_1 P_1 f_1 f^2}{f^2 + f_1^2} + \frac{A_2 P_2 f_2 f^2}{f^2 + f_2^2} + A_3 P_3 f^2$  
$A_1 = (8.86/c) 10^{0.78pH-5}$  
$f_1 = 2.8(S/35)^{0.5} 10^{[4-1245/(T+273)]}$  
$P_1 = 1$  
$A_2 = 21.44(S/c)(1+0.025T)$  
$P_2 = 1-1.37 \times 10^{-4}z + 6.2 \times 10^{-9}z^2$  
$f_2 = 8.17 \times 10^{[8-1990/(T+273)]} /[1 + 0.0018(S-35)]$  
$P_3 = 1-3.83 \times 10^{-5}z + 4.9 \times 10^{-10}z^2$  
$A_3 = \begin{cases}
  4.937 \times 10^{-4} - 2.59 \times 10^{-5}T + 9.11 \times 10^{-7}T^2 - 1.5 \times 10^{-8}T^3, & \text{if } T \le 20^{\circ}C, \\
  3.964 \times 10^{-4} - 1.146 \times 10^{-5}T + 1.45 \times 10^{-7}T^2 - 6.5 \times 10^{-10}T^3, & \text{if } T \gt 20^{\circ}C.
\end{cases}$
- f = frequency in kHz
- T = temp in $^{\circ}$C
- S = salinity in PSU, parts per thousand
- z = depth in meters  
- pH = pH (unitless)
- c = sound speed in m/s
- Claimed to predict $\alpha$ to an accuracy of 5% for temps from -1.8-30$^{\circ}$C, salinities from 30-35ppt, frequencies from 400Hz-1MHz
- pH of sw 7.8-8.2. Reasonable to assume pH=8.
- From Francois and Garrison (1982) from Simmonds and MacLennan (2008)

# References cited

Deines, K. L. (1999). Backscatter estimation using Broadband acoustic Doppler current profilers. In Proceedings of the IEEE Sixth Working Conference on Current Measurement (Cat. No.99CH36331) (pp. 249–253). https://doi.org/10.1109/CCM.1999.755249

Dwinovantyo, A., Manik, H. M., Prartono, T., Susilohadi, S., & Mukai, T. (2019). Variation of Zooplankton Mean Volume Backscattering Strength from Moored and Mobile ADCP Instruments for Diel Vertical Migration Observation. Applied Sciences, 9(9), 1851. https://doi.org/10.3390/app9091851

Francois, R. E., & Garrison, G. R. (1982). Sound absorption based on ocean measurements. Part II: Boric acid contribution and equation for total absorption. The Journal of the Acoustical Society of America, 72(6), 1879–1890. https://doi.org/10.1121/1.388673

Gostiaux, L., & van Haren, H. (2009). Extracting Meaningful Information from Uncalibrated Backscattered Echo Intensity Data. Journal of Atmospheric and Oceanic Technology, 27(5), 943–949. https://doi.org/10.1175/2009JTECHO704.1

Lee, K., Mukai, T., Kang, D., & Iida, K. (2004). Application of acoustic Doppler current profiler combined with a scientific echo sounder for krill Euphausia pacifica density estimation. Fisheries Science, 70(6), 1051–1060. https://doi.org/10.1111/j.1444-2906.2004.00905.x

Leroy, C. C. (1969). Development of Simple Equations for Accurate and More Realistic Calculation of the Speed of Sound in Seawater. The Journal of the Acoustical Society of America, 46(1B), 216–226. https://doi.org/10.1121/1.1911673

Mackenzie, K. V. (1981). Nine‐term equation for sound speed in the oceans. The Journal of the Acoustical Society of America, 70(3), 807–812. https://doi.org/10.1121/1.386920

Mullison, J. (2017). Backscatter Estimation Using Broadband Acoustic Doppler Current Profilers - Updated (FSA No. 031). Poway, CA: Teledyne RD Instruments. Retrieved from http://www.teledynemarine.com/Documents/Brand%20Support/RD%20INSTRUMENTS/Technical%20Resources/Technical%20Notes/WorkHorse%20-%20ADCP%20Special%20Applications%20and%20Modes/FSA031.pdf

Simmonds, J., & MacLennan, D. N. (2008). Fisheries Acoustics: Theory and Practice. John Wiley & Sons.

Teledyne RD Instruments. (1998). FST-003 - Calculating absolute backscatter in narrowband ADCPs (Field Service Technical Paper No. FST-003). Retrieved from http://www.teledynemarine.com/Documents/Brand%20Support/RD%20INSTRUMENTS/Technical%20Resources/Technical%20Notes/Narrowband/FST003.PDF

Teledyne RD Instruments. (2011). Acoustic Doppler Current Profiler Principles of Operation: A Practical Primer (No. P/N 951-6069-00). Retrieved from https://www.comm-tec.com/Docs/Manuali/RDI/BBPRIME.pdf

Urick, R. (1983). Principles of Underwater Sound, Third edition (Soft and Hard Covers). Peninsula Pub.